In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import sys
import os
import io
import shutil

import pudl
import pudl.glue.ferc1_eia
from pudl import init
from pudl import constants as pc
import tableschema
import hashlib
import datapackage
import json
import logging
from tableschema import Table
from tableschema import Schema
import datetime
import goodtables

In [2]:
from pudl import ETL_pkg
from pudl.ETL_pkg import ETL_pkg, _input_validate
from pudl.ETL_pkg import _ETL_eia_pkg, _ETL_ferc1_pkg, _ETL_glue

In [3]:
from pudl.settings import SETTINGS

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [ ]:
from sqlalchemy.engine import reflection
pudl_engine = init.connect_db(testing=False)
insp = reflection.Inspector.from_engine(pudl_engine)
# extract the table names
tbls = insp.get_table_names()
# extract only the ferc tables
ferc_tbls = [s for s in tbls if "ferc" in s]

If you want to generate the package for just ferc as an example, you can use the commented out ferc options below.

In [ ]:
name = "pudl-test"
title = "All tables integrated into PUDL."
tbls = tbls

#name = "pudl-ferc1"
#title =  "FERC Form 1 tables integrated into PUDL."
#tbls = ferc_tbls

In [ ]:
#tbls.remove('hourly_emissions_epacems')

In [ ]:
# we need this as the main info regarding this iteration of packaging
pkg_skeleton = {
    "name": name,
    "title": title,
    "description": "A full metadat description of all PUDL tables.",
    "keywords": [
        "ferc",
        "form 1",
        "energy",
        "electricity",
        "utility",
        "fuel",
        "expenses",
        "coal",
        "natural gas",
        "generation",
        "regulation"
    ]
}

this will generate metadata and csv's for every table in pudl. they will live in `results/data_pkgs`

In [ ]:
pkg = pudl.output.export.data_package(tbls,pkg_skeleton,out_dir=SETTINGS['meta_dir'], dry_run=True)

In [45]:
out_dir = SETTINGS['out_dir']
settings_init = pudl.settings.settings_init(settings_file='settings_init_pudl_package.yml')

In [44]:
metas = pudl.output.export.generate_data_packages(settings_init, debug=True)

glue_pkg
Tables are consistent for glue_pkg package
Not generating metadata for glue_pkg
ferc1_pkg
Not loading FERC1
Tables are consistent for ferc1_pkg package
Not generating metadata for ferc1_pkg
eia_pkg
Loading Static EIA Tables fuel_type_eia923 dataframe into CSV
Loading Static EIA Tables prime_movers_eia923 dataframe into CSV
Loading Static EIA Tables fuel_type_aer_eia923 dataframe into CSV
Loading Static EIA Tables energy_source_eia923 dataframe into CSV
Loading Static EIA Tables transport_modes_eia923 dataframe into CSV
Extracting EIA 923 spreadsheets for 2017.
Converting EIA 923 generation_fuel spreadsheet tab from 2017 into a pandas DataFrame
Converting EIA 923 stocks spreadsheet tab from 2017 into a pandas DataFrame
Converting EIA 923 boiler_fuel spreadsheet tab from 2017 into a pandas DataFrame
Converting EIA 923 generator spreadsheet tab from 2017 into a pandas DataFrame
Converting EIA 923 fuel_receipts_costs spreadsheet tab from 2017 into a pandas DataFrame
Beginning ETL 

/Users/christinagosnell/anaconda/envs/pudl/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Average consistency of static plants values is 100.00%
Harvesting IDs & consistently static attributes for EIA generators
Average consistency of static generators values is 100.00%
Harvesting IDs & consistently static attributes for EIA utilities
Average consistency of static utilities values is 100.00%
Harvesting IDs & consistently static attributes for EIA boilers
Average consistency of static boilers values is 97.58%
Inferring complete EIA boiler-generator associations.
Multiple EIA unit codes:plant_id_eia=10725, unit_id_pudl=1, unit_id_eia=['F801' 'F802']
Multiple EIA unit codes:plant_id_eia=55309, unit_id_pudl=1, unit_id_eia=['SMR1' 'SMR2']
Multiple EIA unit codes:plant_id_eia=60786, unit_id_pudl=1, unit_id_eia=[4343 4141]
Loading Entities plants_entity_eia dataframe into CSV
Loading Entities generators_entity_eia dataframe into CSV
Loading Entities utilities_entity_eia dataframe into CSV
Loading Entities boilers_entity_eia dataframe into CSV
Loading EIA ownership_eia860 dataframe

- Cems_pkg
    - data
        - cems2017.csv.zip
        - cems2016.csv.zip
        - cems2015.csv.zip
        (see "Compression of Resources": http://frictionlessdata.io/specs/patterns/)
    - datapackage.json
        which includes a list of paths in "paths" in the resource
        (see "Data in Multiple Files": https://frictionlessdata.io/specs/data-resource/)


I need to convert the ETL cems process to generate years of cems at a time and write it compressed.

In [12]:
import copy

In [24]:
metas['eia_pkg'][1]

{'time': 5.696,
 'valid': True,
 'error-count': 0,
 'table-count': 10,
 'tables': [{'datapackage': '/Users/christinagosnell/code/pudl/results/data_pkgs/eia_pkg/datapackage.json',
   'time': 4.767,
   'valid': True,
   'error-count': 0,
   'row-count': 1000,
   'source': '/Users/christinagosnell/code/pudl/results/data_pkgs/eia_pkg/data/boiler_fuel_eia923.csv',
   'headers': ['id',
    'plant_id_eia',
    'boiler_id',
    'fuel_type_code',
    'fuel_type_code_pudl',
    'report_date',
    'fuel_consumed_units',
    'fuel_mmbtu_per_unit',
    'sulfur_content_pct',
    'ash_content_pct'],
   'format': 'inline',
   'schema': 'table-schema',
   'errors': []},
  {'datapackage': '/Users/christinagosnell/code/pudl/results/data_pkgs/eia_pkg/datapackage.json',
   'time': 4.04,
   'valid': True,
   'error-count': 0,
   'row-count': 1000,
   'source': '/Users/christinagosnell/code/pudl/results/data_pkgs/eia_pkg/data/boiler_generator_assn_eia860.csv',
   'headers': ['id',
    'plant_id_eia',
    're

In [19]:
#ferc_vali = metas['ferc1_pkg'][1]
#ferc_meta = metas['ferc1-pkg'][0]
#eia_vali = metas['eia_pkg']
cems_vali = metas['epacems_eia860_pkg']

In [20]:
cems_vali[1]

{'time': 4.822,
 'valid': True,
 'error-count': 0,
 'table-count': 10,
 'tables': [{'datapackage': '/Users/christinagosnell/code/pudl/results/data_pkgs/epacems_eia860_pkg/datapackage.json',
   'time': 3.826,
   'valid': True,
   'error-count': 0,
   'row-count': 1000,
   'source': '/Users/christinagosnell/code/pudl/results/data_pkgs/epacems_eia860_pkg/data/boiler_fuel_eia923.csv',
   'headers': ['id',
    'plant_id_eia',
    'boiler_id',
    'fuel_type_code',
    'fuel_type_code_pudl',
    'report_date',
    'fuel_consumed_units',
    'fuel_mmbtu_per_unit',
    'sulfur_content_pct',
    'ash_content_pct'],
   'format': 'inline',
   'schema': 'table-schema',
   'errors': []},
  {'datapackage': '/Users/christinagosnell/code/pudl/results/data_pkgs/epacems_eia860_pkg/datapackage.json',
   'time': 3.41,
   'valid': True,
   'error-count': 0,
   'row-count': 1000,
   'source': '/Users/christinagosnell/code/pudl/results/data_pkgs/epacems_eia860_pkg/data/boiler_generator_assn_eia860.csv',
   '

In [9]:
ferc_vali

{'time': 0.042,
 'valid': True,
 'error-count': 0,
 'table-count': 0,
 'tables': [],
 'warnings': ['Data Package "/Users/christinagosnell/code/pudl/results/data_pkgs/ferc1_pkg/datapackage.json" has a validation error "Descriptor validation error: [] is too short at "resources" in descriptor and at "properties/resources/minItems" in profile"'],
 'preset': 'datapackage'}

In [ ]:
metas.keys()

In [ ]:
cems_vali = metas['epacems_eia860_pkg']

In [ ]:
cems_vali

In [26]:
pc.tab_map_eia923.columns

Index(['generation_fuel', 'stocks', 'boiler_fuel', 'generator',
       'fuel_receipts_costs', 'plant_frame'],
      dtype='object')

In [28]:
pc.pudl_tables['eia923']

('generation_fuel_eia923',
 'boiler_fuel_eia923',
 'generation_eia923',
 'coalmine_eia923',
 'fuel_receipts_costs_eia923')

- the main coordinating function is pudl.output.export.generate_data_packages()
    - uses settings:
        -yml file (ex: ‘settings_init_pudl_package.yml’) and settings.py
    - validates settings using `pudl.ETL_pkg._input_validate`... this spits out a new, validated list of package settings
    - then for each of the of the packages definied in settings, run the ETL_pkg function which runs a data source specific function for each dataset. The ETL_pgk function needs the settings/inputs. In a stand-alone version of this function these can be validated settings or non-validated settings. The settings will get re-validated immediately inside of the data source specific ETL function. These ETL functions generate CSVs in the out_dir/(package name)/data folder.
    - an output of the ETL_pkg function is a list of tables being generated. this list is piped through to a `test_file_consistency` function, which ensures the ETL tables, the CSVs and dependencies from the metadata
    - then the `generate_metadata` function is run. at a high level, this generates and validates the `datapackage.json` file and runs `goodtables.validate`.